In [1]:
import pysindy as ps

import deepSI
from deepSI import System_data
from deepSI.fit_systems import SS_encoder_general
from deepSI.fit_systems.encoders import default_encoder_net, default_state_net, default_output_net
from deepSI.fit_systems.fit_system import My_Simple_DataLoader, print_array_byte_size, Tictoctimer
import torch
from torch import nn

import numpy as np

from sklearn.preprocessing import PolynomialFeatures

from scipy.io import loadmat
import scipy.linalg as lin
import os

from utils import load_data
from data_processing import add_gaussian_noise

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import time
from copy import deepcopy
import itertools

import re

from feature_construction import feature_library
import polynomial as p
import fourier as f

from SI_SUBNET import SS_encoder_general_eq, h_identity, e_identity, simple_Linear
from SI_metrics import plot_coeff_grid, display_equation
from run_SISUBNET import run_SISUBNET

import random
import string

from scipy.io import loadmat, savemat
import json
import pickle

c:\Users\Joost\anaconda3\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nf = [1, 5, 10, 25, 50]
gamma = 2e-3
epo_idx = 300/10

pruning = True
mode = None
T_idx = None

batch_size = 256
epochs = 300
no_reg_epochs = 100
pre_train = True

threshold = 1e-3

In [3]:
data_set_name="sc_arrow_noisy"
x_data, u_data, y_data, eq, T, U, set_idx = load_data(set=data_set_name)
[N_arrow, N_train, N_val, N_test] = set_idx[0]

if T is None:
    T = np.eye(x_data.shape[-1])
if U is None:
    U = np.eye(u_data.shape[-1])

# data organizing
x_arrow, u_arrow = x_data[:N_arrow], u_data[:N_arrow,0]
x_train, u_train = x_data[N_arrow:N_arrow+N_train-N_val,:], u_data[N_arrow:N_arrow+N_train-N_val,0]
x_val, u_val     = x_data[N_arrow+N_train-N_val:-N_test,:], u_data[N_arrow+N_train-N_val:-N_test,0]
x_test, u_test   = x_data[-N_test:,:], u_data[-N_test:]

# system data
arrow   = System_data(u=u_arrow, y=x_arrow)
train   = System_data(u=u_train, y=x_train)
val     = System_data(u=u_val[:1000], y=x_val[:1000,:])
test    = System_data(u=u_test, y=x_test)

# function lib
functions = [p.f, f.sin, f.cos, f.sin2x, f.cos2x, f.sin3x, f.cos3x, f.sin4x, f.cos4x, f.sin5x, f.cos5x]
            #  f.sin11x, f.cos11x, f.sin12x, f.cos12x, f.sin13x, f.cos13x, f.sin14x, f.cos14x, f.sin15x, f.cos15x, f.sin16x, f.cos16x, f.sin17x, f.cos17x, f.sin18x, f.cos18x, f.sin19x, f.cos19x, f.sin20x, f.cos20x]
poly = feature_library(functions=functions, nx=2, nu=1, T=T, U=U)

# network param
nx, nu = 2, 1 # state dimension and inputs
na, nb = 0, 0
na_right = 1

f_net_kwargs = {"feature_library": poly, "u": nu, "nf": poly.feature_number()}

# output model
h_net = h_identity
h_net_kwargs = {}

# encoder model
e_net = e_identity
e_net_kwargs = {}

Data loaded from sc_arrow_noisy!


In [4]:
def convert_float_to_string(number):
    coefficient, exponent = f"{number:.1e}".split('e')
    return f"{int(float(coefficient))}em{-int(exponent)}"

In [5]:
for nf_i in nf:
    set_name = "nftest_org_v2{}".format(int(nf_i))
    print(set_name)

    run_SISUBNET(nx, na, nb, na_right, \
             f_net_kwargs, e_net, e_net_kwargs, h_net, h_net_kwargs,\
             arrow, train, val, test, data_set_name,\
             nf_i, gamma, mode, T_idx, pruning, threshold, epo_idx,\
             batch_size, pre_train=pre_train, no_reg_epochs=no_reg_epochs, nf_epochs=epochs,\
             save_data=True, set_name=set_name, verbose=True)

nftest_org_v250
start
Initilizing the model and optimizer
Size of the training array =  11.0 MB
N_training_samples = 179999, batch_size = 4096, N_batch_updates_per_epoch = 43
Initial Validation sim-NRMS= 2.7296694971638233


  1%|          | 1/100 [00:01<02:27,  1.49s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 1.9136788925153354
Epoch    1, sqrt loss  0.2099, Val sim-NRMS  1.914, Time Loss: 19.8%, data: 0.9%, val: 79.3%, 28.9 batches/sec


  2%|▏         | 2/100 [00:02<02:26,  1.50s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 1.3967043597972228
Epoch    2, sqrt loss  0.1352, Val sim-NRMS  1.397, Time Loss: 19.8%, data: 1.0%, val: 79.2%, 28.7 batches/sec


...

 99%|█████████▉| 99/100 [34:29<00:15, 15.11s/it]

Epoch   99, sqrt loss 0.01492, Val sim-NRMS 0.09581, Time Loss: 92.6%, data: 0.3%, val: 7.2%,  2.1 batches/sec


100%|██████████| 100/100 [34:44<00:00, 20.84s/it]

Epoch  100, sqrt loss 0.01492, Val sim-NRMS 0.09593, Time Loss: 92.6%, data: 0.3%, val: 7.2%,  2.1 batches/sec


arrow test complete!
test test complete!
arrow test complete!
test test complete!
done


In [8]:
torch.ones(5,2) == 1

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True]])